BIBLIOTECAS

In [38]:
import pandas as pd
from sqlalchemy import create_engine
import datetime

EXTRAÇÃO DOS DADOS

In [39]:
# Dados de acesso PostgreSQL
trt_name = 'automatiza_metricas_dev'
trt_user = 'automatiza'
trt_password = 'automatiza'
trt_host = '10.5.15.78'
trt_port = '5432'

# Acessa o banco de dados e cria a engine
db_url = f'postgresql+psycopg2://{trt_user}:{trt_password}@{trt_host}:{trt_port}/{trt_name}'
engine = create_engine(db_url)

# Seleciona as tabelas
query_cartoes = "SELECT * FROM cartoes"
query_checklists = "SELECT * FROM checklists"
query_listas = "SELECT * FROM listas"
query_quadros = "SELECT * FROM quadros"

# Armazena tabelas em dataframes
df_cartoes = pd.read_sql(query_cartoes, engine)
df_checklists = pd.read_sql(query_checklists, engine)
df_listas = pd.read_sql(query_listas, engine)
df_quadros = pd.read_sql(query_quadros, engine)

# Copia de segurança das tabelas
quadros = df_quadros.copy()
listas = df_listas.copy()
cartoes = df_cartoes.copy()
checklists = df_checklists.copy()

TRANSFORMAÇÃO DOS DADOS

In [40]:
# Renomear colunas das tabelas
quadros.rename(columns={'nome': 'quadro'}, inplace=True)
listas.rename(columns={'nome': 'lista'}, inplace=True)
cartoes.rename(columns={'nome': 'cartao'}, inplace=True)
checklists.rename(columns={'nome': 'checklist'}, inplace=True)

# Tratar valores nulos na tabela Cartoes
cartoes['data_inicio'] = cartoes['data_inicio'].fillna(cartoes['data_fim'])
cartoes['data_fim'] = cartoes['data_fim'].fillna(cartoes['data_inicio'])
cartoes[['data_inicio', 'data_fim']] = cartoes[['data_inicio', 'data_fim']].fillna(pd.Timestamp('01-01-1900'))
cartoes['lead_time'] = cartoes['lead_time'].fillna(0)
cartoes['descricao'] = cartoes['descricao'].replace('', 'SEM DESCRICAO')

# Converter formato de datas na tabela Cartoes
cartoes['data_inicio'] = pd.to_datetime(cartoes['data_inicio'], format='%d-%m-%Y')
cartoes['data_fim'] = pd.to_datetime(cartoes['data_fim'], format='%d-%m-%Y')

# Converter tipo de dados na tabela Cartoes \lead_time
cartoes['lead_time'] = cartoes['lead_time'].astype(int)

# Consolidar tabelas
consolidado = pd.concat([checklists, cartoes, listas, quadros], axis=1)



In [41]:
display(consolidado.head(3))

,id_checklist,checklist,quant_itens,id_cartao,id_cartao,cartao,descricao,data_inicio,data_fim,lead_time,id_lista,id_lista,lista,id_quadro,id_quadro,quadro
0,65dcbad8c8ea9de6a3f1fce6,Checklist,1.0,65dcbac6e91e9bacf68c6090,661011d5b8ec5c93ba4de7ac,Criar uma tela de consulta pública para todas ...,**Como** Usuário externo (advogado ou procurad...,2024-05-08,2024-07-31,84,65f1e8e0cd957bafc99eada0,65f1e79cce8368236efc69bc,CONCLUÍDO,65f1e79cce8368236efc69b7,5606128cefdfd254db45d43d,Videoteca
1,653bbe059a4aec0bb6b5c5c5,Checklist,1.0,653bbdf1e810eddf96f1dcec,6672e1a27fb363377b84a4d6,[ADM] Atualização de Secrets: não está conside...,SEM DESCRICAO,2024-07-22,2024-08-12,21,64b58c9ba9edbfbe62d3a8bd,66bb5d75a8d4e23c398232a7,concluído,66b2308d8280fee1577029f4,65f1e79cce8368236efc69b7,AutomatizaTRT
2,64e8abe8db6a47579d951e8a,Checklist,5.0,64e8abdfa1f13416e6edbcf6,632de9ba9a8f070108a20d3f,PJEKZ-32845 > PJEKZ-71081[Remessa] Baixa de pr...,Aguardando versão alvo para encaminharmos para...,2024-03-05,2024-03-05,0,632de9ba9a8f070108a20ba1,64b58c9ba9edbfbe62d3a8bd,Concluído,6076ee3e1333ca13c49be9f9,6076ee3e1333ca13c49be9f9,JT SIGEP
